# L0 data clean

In [1]:
import imp
import settings

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import imp
import datetime
import pytz
%matplotlib notebook

In [3]:
#import self-written libraries
import LVL1
import settings
from settings import Glacier, Station, base_path

## Define Glacier and Station

In [4]:
Glacier='Wolverine'
Station='990'

data_columns, general_data_columns, out_date_format, precip_columns, precip_gage_change, primary_temp_column, temp_columns, timezone, wind_col, wind_dir_columns = settings.get_settings(settings.Glacier, settings.Station)

In [5]:
#Are there newly downloaded data?
new_data=False

In [6]:
if new_data:
    add_new_logger_data_script_pth='"' + base_path + 'Code/Weather/LVL_0/ConcatenateRawWxData.py'+ '"'
    %run $add_new_logger_data_script_pth

In [7]:
if (Glacier + Station =='Wolverine370')| (Glacier =='SouthCascade'):
    print ('STOP: not needed here; start at L1. All data from NWIS')
    print('Ensure that Add1990sDataFromNWIS.py has been run')

In [8]:
if (Glacier + Station== 'Wolverine1420') | (Glacier + Station== 'Gulkana1920'): #If a JWS station, which logs @ 5 minutes
    if new_data:
        convert5minto15min_script_pth='"' + base_path + 'Code/Weather/LVL_0/ConvertFrom5minTo15minData_JWS.py'+ '"'

        %run $convert5minto15min_script_pth

In [9]:
#At the Wolverine high JWS site, add TPG (was logged internally, never wired in to logger direclty before moving).
if (Glacier + Station== 'Wolverine1420'): #add TPG log to the high wolverine station, during the time which it existed there.
    addTPGdata_script_pth='"' + base_path + 'Code/Weather/LVL_0/AddWolverineTPGData.py'+ '"'
    
    %run $addTPGdata_script_pth

In [10]:
#Read in data
folder=base_path+ r"Data/" + Glacier+ "/AllYears/Wx/Raw" #folder cont
file_label='_15min_all' #used for importting and output name
file=Glacier.lower()+Station+file_label+".csv" #filename; change if yours different
pth=os.path.join(folder, file) #path to data file

In [11]:
#Set constants for input data
date_format='%m/%d/%Y %H:%M:%S' #input date format (change if yours is different)

In [12]:
dat=pd.read_csv(pth)
print("read file: " + pth)
dat.head()

read file: Q:/Project Data/GlacierData/Benchmark_Program/Data/Wolverine/AllYears/Wx/Raw\wolverine990_15min_all.csv


C:\Users\ehbaker\AppData\Local\Continuum\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Date,Time,Instrument,Value,Unit,Flag,DateTime
0,03/01/2010,23:15:00,WindDir,900.0,Deg,G,03/01/2010 23:15:00
1,03/01/2010,23:15:00,PrecipStage,0.0,M,B,03/01/2010 23:15:00
2,03/01/2010,23:15:00,WindSpeed,0.9,m/s,G,03/01/2010 23:15:00
3,03/01/2010,23:28:00,WindSpeed,0.4,m/s,G,03/01/2010 23:28:00
4,03/01/2010,23:28:00,WindDir,914.0,Deg,G,03/01/2010 23:28:00


## Convert Logger time to UTC and Local Time

In [16]:
#Logger records on UTC -8; convert to local time
#Create date-time column
#dat['DateTime']=dat.Date+ " " + dat.Time #this is needed for non-cumulative processing
dat.loc[:,'DateTime']=pd.to_datetime(dat['DateTime'], format=date_format) #set to date-time from string

In [17]:
dat.head()

,DateTime,UTC_time,Tpassive1,WindSpeed,WindDir,RelHum,SWup_incoming,SWdn_outgoing,LWup_incoming,LWdn_outgoing
0,2006-06-28 14:00:00,2006/06/28 20:00,13.480,4.05400,245.90,51.93,NaN,NaN,NaN,NaN
1,2006-06-28 14:15:00,2006/06/28 20:15,13.625,4.35025,248.42,49.90,NaN,NaN,NaN,NaN
2,2006-06-28 14:30:00,2006/06/28 20:30,13.770,4.64650,250.95,47.87,NaN,NaN,NaN,NaN
3,2006-06-28 14:45:00,2006/06/28 20:45,13.915,4.94275,253.48,45.84,NaN,NaN,NaN,NaN
4,2006-06-28 15:00:00,2006/06/28 21:00,14.060,5.23900,256.00,43.81,NaN,NaN,NaN,NaN


In [18]:
#Correct for time slips
dat['DateTime']=dat['DateTime'].dt.round('15min') #round time to the nearest 15 minute value

#Convert to UTC time, if logger is logging in UTC-8 (true at some stations)
if (Glacier+Station=='Wolverine990') | (Glacier+Station=='Gulkana1480') | (Glacier+Station=='Gulkana1920') | (Glacier+Station=='Wolverine1420'):
    dat['UTC_col']=dat.DateTime + datetime.timedelta(hours=8)
if (Glacier+Station=='Gulkana1725'):
    dat['UTC_col']=dat.DateTime #logger is already in UTC time
if Glacier + Station=='Sperry2440':
    dat['UTC_col']=dat.UTC_time.copy()

In [19]:
dat['UTC_col']=pd.to_datetime(dat['UTC_col'], format=date_format)

In [20]:
#Set timezone of known UTC column
dat['UTC_col'].timezone='UTC'
#Create column for local time
local_timezone=pytz.timezone(timezone) #create local timezone object (e.g. AK time)

In [21]:
dat=dat.set_index('UTC_col')
#Create column for local time
local_timezone=pytz.timezone(timezone) #create local timezone object (e.g. AK time)

#Create local time
dat['local_time'] = dat.index.tz_localize('UTC').tz_convert(local_timezone)

dat['index_local_time']=dat.local_time.copy()

#Set index to local time
dat=dat.set_index('index_local_time')

In [22]:
#Drop duplicates that have same instrument name and date/time stamp (from multiple downloads of same portion of data)
dat=dat.drop_duplicates(subset=['DateTime', 'Instrument'], keep='first') #gets rid of overlaps (i.e. multiple downloads of same bit of data)

KeyError: 'Instrument'

In [ ]:
dat.head(1)

In [ ]:
dat.Value[dat.Instrument=='AirTemp'].plot()

In [ ]:
#Instrument Label Change - if same label has been used for different quantities thru time: separate into 2 distinct labels.
if Glacier + Station=="Wolverine990":
    dat.loc[(dat.Instrument=='WD') & (dat.index<'2015-09-06 17:15'), 'Instrument']='WindDir' #before this date, "WD" was simply wind direction; after, is vector-averaged
    dat.loc[(dat.Instrument=='WD') & (dat.index>='2015-09-06 17:15'), 'Instrument']='WDSA' 
    
if Glacier + Station=="Gulkana1480":
    dat.loc[(dat.Instrument=='WD') & (dat.index<'2015-09-28 12:00'), 'Instrument']='WindDir' #before this date, "WD" was simply wind direction; after, is vector-averaged
    dat.loc[(dat.Instrument=='WD') & (dat.index>='2015-09-28 12:00'), 'Instrument']='WDSA' 
    
#Gulkana 1725 has had consistent variable naming throughout

## Consolidate Sensor Naming
#### Has varied thru time

In [ ]:
#If site does NOT have a given sensor, leave label, create empty list

if Glacier + Station =='Wolverine990':  
    #Name recorded on the logger changed thru time; indicate which are grouped (i.e. logging same variable) here
    wind_gust_labels=['WindGusts', 'WG', 'WSG']
    wind_speed_labels=['AvgWind', 'WS'] #this is NOT vector-averaged wind speed
    vec_avg_wind_dir=['WDSA']
    wind_dir_labels=['WindDir']
    temp_unasp1_labels=['AirTemp1', 'T1', 'T']
    temp_unasp2_labels=['AirTemp2', 'T2']
    temp_asp1_labels=['AirTempAsp', 'AspT', 'AT']
    temp_asp2_labels=['ASPTRH', 'AT2']
    precip_stage_labels=['PR']
    precip_weighing_labels=['TPG', 'PC']
    precip_incremental_labels=[]
    rel_hum_labels=['RH']
    barometer_labels=['B', 'BAROMETER']
    logger_bat_label=['BV']
    logger_temp_label=['Tinternal']
    bad_labels=['PrecipStage', 'AirTemp3', 'Air Temp w/FanN', 'AirTemp w/Fan', 'VectorAvgWindDir', 'PS', 'MWD', 'WDA'] #labels containing only bad data; introduced during maintenence etc.
    radiation_up_labels=[]
    radiation_down_labels=[]
    snow_depth_labels=[] 
    
if Glacier + Station =='Gulkana1480':  
    #Name recorded on the logger changed thru time; indicate which are grouped (i.e. logging same variable) here
    wind_gust_labels=['WSG']
    wind_speed_labels=['WindSpeed', 'WS']
    vec_avg_wind_dir=['WDSA']
    wind_dir_labels=['WindDir'] #this is NOT-vector average wind direction
    temp_unasp1_labels=['AirTemp1', 'T1', 'T']
    temp_unasp2_labels=['AirTemp2', 'T2']
    temp_asp1_labels=['AspT', 'AT']
    temp_asp2_labels=['AT2']
    precip_stage_labels=['PR'] #precip doesn't begin until 2010-07; before this, is recording, but logger only has 0 precip
    precip_weighing_labels=['TPG', 'PC']
    precip_incremental_labels=[]
    rel_hum_labels=['RH']
    barometer_labels=[] #no barometer at this station
    logger_bat_label=['BV']
    logger_temp_label=['Tinternal']
    bad_labels=['PrecipStage', 'AirTemp3', 'Stage'] #labels containing only bad data; introduced during maintenence etc.
    radiation_up_labels=[]
    radiation_down_labels=[]
    snow_depth_labels=[]    
    
if Glacier + Station=='Gulkana1725':
    #Only a single name thru time; no changes @ 1725
    wind_gust_labels=[]#no wind gust recorded
    wind_speed_labels=['MS']
    vec_avg_wind_dir=['MD']
    wind_dir_labels=[] #this is NOT-vector average wind direction
    temp_unasp1_labels=['TA'] #TA is NOT Aspriated; it's Temperature, Air
    temp_unasp2_labels=['T']
    temp_asp1_labels=[]
    temp_asp2_labels=[]
    precip_stage_labels=[]
    precip_weighing_labels=['PC']
    precip_incremental_labels=[]
    rel_hum_labels=[]
    barometer_labels=[] #no barometer at this station
    logger_bat_label=['VB']
    logger_temp_label=[]
    bad_labels=[]
    radiation_up_labels=['RU']
    radiation_down_labels=['RD']
    snow_depth_labels=['DS']
    
if (Glacier + Station=='Gulkana1920') | (Glacier + Station=='Wolverine1420'): #same program and same variable naming at JWS stations
    wind_gust_labels=[]#no wind gust recorded
    wind_speed_labels=['WS_ms_S_WVT']
    vec_avg_wind_dir=['WindDir_D1_WVT'] #I am guessing this is vec acerage; should come back and check program before using
    wind_dir_labels=[] #this is NOT-vector average wind direction
    temp_unasp1_labels=['AirTempC_Avg']
    temp_unasp2_labels=[]
    temp_asp1_labels=[]
    temp_asp2_labels=[]
    precip_stage_labels=[]
    precip_weighing_labels=[]
    precip_incremental_labels=[]
    if Station=='Wolverine1420':#only true at Wolverine
        precip_weighing_labels=['TPGCumulative']
    rel_hum_labels=['RH']
    barometer_labels=['BP_inHg_Avg']
    logger_bat_label=['BattV_Avg']
    logger_temp_label=[]
    bad_labels=[]
    radiation_up_labels=['PyrnUp_Avg']
    radiation_down_labels=['PyrnDn_Avg']
    snow_depth_labels=[]

    
    #COME BACK AND START HERE AFTER FIELDWORK WITH AVO!!
if Glacier =='JuneauIcefield' and "AWS" in Station: #differentiate between 3 AWS stations, vs. jirp hobo loggers
    wind_gust_labels=['WindSpMax', 'WindSpGust', 'WindSP_Gust']#no wind gust recorded
    wind_speed_labels=['WS_ms_S_WVT', 'WindSpeed', 'WindSp', 'WindSP']
    vec_avg_wind_dir=['WindDir_D1_WVT', 'WindDir_VecAvg'] #I am guessing this is vec acerage; should come back and check program before using
    wind_dir_labels=['WindDir'] #this is NOT-vector average wind direction
    temp_unasp1_labels=['AirTC_Avg', 'AirTemp']
    temp_unasp2_labels=[]
    temp_asp1_labels=[]
    temp_asp2_labels=[]
    precip_stage_labels=[]
    precip_weighing_labels=[]
    precip_incremental_labels=['Precip_m']
    rel_hum_labels=['RH']
    barometer_labels=['BP_mbar', 'BP_kPa_Avg', 'Baro_Kpa', 'Baro']
    logger_bat_label=['BattV_Avg']
    logger_temp_label=[]
    bad_labels=['Time']
    radiation_up_labels=['PyrnUp_Avg', 'SolRad']
    radiation_down_labels=['PyrnDn_Avg']
    snow_depth_labels=['TCDT', 'SnowDep1']


In [ ]:
dat.Instrument.unique()

In [ ]:
#Rename instruments 
dat.loc[dat.Instrument.isin(wind_gust_labels), "Instrument"]='WindGustSpeed'
dat.loc[dat.Instrument.isin(wind_speed_labels), "Instrument"]='WindSpeed'
dat.loc[dat.Instrument.isin(wind_dir_labels), "Instrument"]='WindDir'
dat.loc[dat.Instrument.isin(vec_avg_wind_dir), "Instrument"]='VecAvgWindDir'
dat.loc[dat.Instrument.isin(temp_unasp1_labels), "Instrument"]='Tpassive1'
dat.loc[dat.Instrument.isin(temp_unasp2_labels), "Instrument"]='Tpassive2'
dat.loc[dat.Instrument.isin(temp_asp1_labels), "Instrument"]='TAspirated1'
dat.loc[dat.Instrument.isin(temp_asp2_labels), "Instrument"]='TAspirated2'
dat.loc[dat.Instrument.isin(precip_stage_labels), "Instrument"]='StageCumulative'
dat.loc[dat.Instrument.isin(precip_weighing_labels), "Instrument"]='TPGCumulative'
dat.loc[dat.Instrument.isin(precip_incremental_labels), "Instrument"]='IncrementalPrecip'
dat.loc[dat.Instrument.isin(rel_hum_labels), "Instrument"]='RelHum'
dat.loc[dat.Instrument.isin(barometer_labels), "Instrument"]='Barom'
dat.loc[dat.Instrument.isin(logger_bat_label), "Instrument"]='LoggerBattery'
dat.loc[dat.Instrument.isin(logger_temp_label), "Instrument"]='LoggerTemp'
dat.loc[dat.Instrument.isin(radiation_up_labels), "Instrument"]='RadiationIn'
dat.loc[dat.Instrument.isin(radiation_down_labels), "Instrument"]='RadiationOut'
dat.loc[dat.Instrument.isin(snow_depth_labels), "Instrument"]='SnowDepth'

#Remove rows with bad data
dat=dat[~dat.Instrument.isin(bad_labels)] #Remove where labels are bad; introduced during maintenance

In [ ]:
print(dat.Instrument.unique())
dat.head()

In [ ]:
#Cast the data from long -> wide format
#dat=dat.reset_index() #reset index so all unique
wide_dat=dat.pivot_table(columns='Instrument', values='Value', index='local_time')

In [ ]:
wide_dat.tail(1)

In [ ]:
plt.figure()
wide_dat.Tpassive1.plot()

In [ ]:
#List columns desired in output
# out_columns=['UTC_time', 'local_time', 'Tpassive1', 'Tpassive2',
#        'TAspirated1', 'TAspirated2', 'RelHum', 'StageCumulative',
#        'TPGCumulative', 'WindSpeed', 'WindGustSpeed', 'WindDir', 'Barom',
#         'VecAvgWindDir', 'RadiationIn', 'RadiationOut', 'SnowDepth', 'LoggerTemp', 'LoggerBattery']

In [ ]:
#This should NO LONGER be neccessary (~4/18/18); future processing can eliminate the all NAN columns
#If this columns is not in dataframe, create
# for col in out_columns:
#     if col not in wide_dat.columns:
#         wide_dat[col]=np.nan #create the column; fill with NANs

In [ ]:
#Reindex to 15min to ensure no timesteps are skipped
wide_dat.sort_index(inplace=True)
full_range_15_min = pd.date_range(wide_dat.index[0], wide_dat.index[-1], freq='15min')

In [ ]:
wide_dat.index[-4]

In [ ]:
full_range_15_min[-4]

In [ ]:
wide_dat=wide_dat.reindex(index=full_range_15_min, fill_value=pd.np.nan)

In [ ]:
#Set output format of time
wide_dat['UTC_time']=wide_dat.index.tz_convert('UTC').strftime(out_date_format)#Create column for true local time (as string, not UTC - X hrs)
wide_dat['local_time']=wide_dat.index.tz_convert(timezone).strftime(out_date_format)#Create column for true local time (as string, not UTC - X hrs)

In [ ]:
wide_dat=wide_dat.reset_index(drop=True).copy() #drop index in output dataframe

In [ ]:
save_dat=wide_dat

In [ ]:
#directory to save output data
save_pth=base_path+ r"Data/" +Glacier+ r"/AllYears/Wx/LVL0/"
out_nm=Glacier.lower()+ Station + file_label+ "_LVL0.csv"
out_nm

In [ ]:
save_dat.head(1)

In [ ]:
#Set a nice column order to save
#out_cols=['local_time', 'UTC_time']+temp_columns + precip_columns +[wind_col] +wind_dir_columns + list(set(save_dat.columns) - set(['local_time', 'UTC_time'] + temp_columns + precip_columns +[wind_col] +wind_dir_columns))

In [ ]:
save_dat[['local_time', 'UTC_time'] +data_columns].to_csv(os.path.join(save_pth, out_nm), index=False, float_format='%g')
print( "saved to: ", save_pth+out_nm)

In [ ]:
#If this is the Nunatak station, fill gaps with Irridium telemetry
if Glacier + Station=='Gulkana1725':
    %run "./Gulkana1725_FillGapsWithIrridium.py"

In [ ]:
#If Wolv 990 or Gulkana 1480, add 1990s data from NWIS to extend record back in time
if (Glacier + Station=='Wolverine990') | (Glacier + Station=='Gulkana1480'):    
    %run "./Add1990sDataFromNWIS.py"

In [ ]:
print('hey')

In [ ]:
save_dat[['local_time', 'UTC_time'] +data_columns]